Xarray processing stream for ERA5 netcdf files
Written by: Matt Jolly
Date: 24 July 2019

In [4]:
%matplotlib inline
import math
import xarray as xr
from dask.diagnostics import ProgressBar
import numpy as np
import pandas as pd
import netCDF4
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [5]:
# Basic file path to NetCDF data files
fpath = '/media/BigFire/projects/era5/data'
# Air temperature file
t2m = 'era5_single_levels_t2m_2018.nc'
t2mfullpath = "%s/%s" % (fpath,t2m)
# Dewpoint temperature file
tdew = 'era5_single_levels_2m_dewpoint_temperature_2018.nc'
tdewfullpath = "%s/%s" % (fpath,tdew)
ofile = tdewfullpath.replace("era5_single_levels_t2m_","era5_2m_daily_relative_humidity_")
print(t2mfullpath)
ds_t2m = xr.open_dataset(t2mfullpath,chunks={'time': 20})
ds_tdew = xr.open_dataset(tdewfullpath,chunks={'time': 20})

/media/BigFire/projects/era5/data/era5_single_levels_t2m_2018.nc


In [6]:
ds_t2m
ds_tdew
ds_tdew.d2m.attrs

OrderedDict([('units', 'K'), ('long_name', '2 metre dewpoint temperature')])

In [ ]:
t2mvp = np.exp(1.81 + (ds_t2m.t2m * 17.27 - 4717.31) / (ds_t2m.t2m - 35.86))
tdewvp = np.exp(1.81 + (ds_tdew.d2m * 17.27 - 4717.31) / (ds_tdew.d2m - 35.86))
rh = (tdewvp / t2mvp) * 100

In [ ]:
# Create an output dataset with all attributes
outds = ds_t2m
outds = outds.drop('t2m')
outds.attrs['Desc'] = "Derived from 2m air temperature and 2m dewpoint temperature from ERA5"
rh.attrs['units'] = "%"
rh.attrs['long_name'] = "2 metre relative humidity"
outds['rh'] = rh
outds = outds.groupby('time.dayofyear').mean('time')
outds.to_netcdf("/run/media/mjolly/BigFire/projects/era5/data/testout.nc",format= 'NETCDF4_CLASSIC' )
outds

In [ ]:
outds.rh.sel(dayofyear=slice('180','240')).mean().shift(longitude=720).plot()

In [ ]:
#Aggregate grid from hourly to daily
r = outds.rh.groupby('time.dayofyear').mean('time')

# Plot a time series by lat / lon
r.sel(latitude=45, longitude=360-114, method='nearest', tolerance=5).plot()

In [ ]:
def CalcVP(TempK):
    # Purpose: Calculate the sautration vapor pressure
    return np.exp(1.81 + (TempK * 17.27 - 4717.31) / (TempK - 35.86))

def CalcVP_XR(a):
    return xr.apply_ufunc(CalcVP, a,keep_attrs=True,dask='parallelized')

#with ProgressBar():
#    ds.load()

In [ ]:
def CVP(TempK):
    func = lambda x: np.exp(1.81 + (x* 17.27 - 4717.31) / (x- 35.86))
    return xr.apply_ufunc(func,TempK,dask='parallelized',output_dtypes=[float])

In [ ]:
ds_sub = ds.sel(time=slice('2018-07-01','2018-07-15'))
ds_sub = ds

In [ ]:
tempvp = CVP(ds_t2m.t2m )

In [ ]:
ds_sub = math.exp(1.81 + (ds_sub.t2m * 17.27 - 4717.31) / (ds_sub.t2m - 35.86))

In [ ]:
#tempvp.sel(time=slice('2018-07-01','2018-07-15')).mean().plot()
tempvp.sel(time=slice('2018-05-01','2018-07-31')).sel(latitude=45, longitude=360-114, method='nearest', tolerance=5).plot()

In [ ]:
d = ds_sub.max(dim='time')

In [ ]:
d.plot()

In [ ]:
ds_sub.sel(latitude=45, longitude=360-114, method='nearest', tolerance=5).plot()


In [ ]:
ds_sub.t2m.attrs

In [ ]:
from netCDF4 import Dataset
dataset = Dataset(t2mfullpath) 
print (dataset.file_format )
print (dataset.dimensions.keys)
print (dataset.dimensions)
print (dataset.variables)

In [ ]:
ds_tdew.d2m.isel(time=0).plot()

In [ ]:
print(CalcVP(290))


In [ ]:
def ProcessERA5RelativeHumidityMin():
    yrs = range(1979,2019)
    for y in yrs:
        print(y)
        # Basic file path to NetCDF data files
        fpath = '/run/media/mjolly/BigFire/projects/era5/data'
        # Air temperature file
        t2m = 'era5_single_levels_t2m'
        t2mfullpath = "%s/%s_%s.nc" % (fpath,t2m,str(y))
        # Dewpoint temperature file
        tdew = 'era5_single_levels_2m_dewpoint_temperature'
        tdewfullpath = "%s/%s_%s.nc" % (fpath,tdew,str(y))
        ofile = tdewfullpath.replace("era5_single_levels_t2m_","era5_2m_daily_relative_humidity_")
        print(t2mfullpath)
        ds_t2m = xr.open_dataset(t2mfullpath,chunks={'time': 20})
        ds_tdew = xr.open_dataset(tdewfullpath,chunks={'time': 20})
        t2mvp = np.exp(1.81 + (ds_t2m.t2m * 17.27 - 4717.31) / (ds_t2m.t2m - 35.86))
        tdewvp = np.exp(1.81 + (ds_tdew.d2m * 17.27 - 4717.31) / (ds_tdew.d2m - 35.86))
        rh = (tdewvp / t2mvp) * 100
        # Create an output dataset with all attributes
        outds = ds_t2m
        outds = outds.drop('t2m')
        outds.attrs['Desc'] = "Derived from 2m air temperature and 2m dewpoint temperature from ERA5"
        rh.attrs['units'] = "%"
        rh.attrs['long_name'] = "2 metre relative humidity"
        outds['rh'] = rh
        outds = outds.groupby('time.dayofyear').min('time')
        ofile = t2mfullpath.replace('t2m','rhmin')
        print("Outfile: %s:" % (ofile))
        outds.to_netcdf(ofile,format= 'NETCDF4_CLASSIC' )
                
        
ProcessERA5RelativeHumidity()
    

In [8]:
def ProcessERA5RelativeHumidityMax():
    yrs = range(2018,1978,-1)
    for y in yrs:
        print(y)
        # Basic file path to NetCDF data files
        fpath = '/media/BigFire/projects/era5/data'
        # Air temperature file
        t2m = 'era5_single_levels_t2m'
        t2mfullpath = "%s/%s_%s.nc" % (fpath,t2m,str(y))
        # Dewpoint temperature file
        tdew = 'era5_single_levels_2m_dewpoint_temperature'
        tdewfullpath = "%s/%s_%s.nc" % (fpath,tdew,str(y))
        ofile = tdewfullpath.replace("era5_single_levels_t2m_","era5_2m_daily_relative_humidity_")
        print(t2mfullpath)
        ds_t2m = xr.open_dataset(t2mfullpath,chunks={'time': 20})
        ds_tdew = xr.open_dataset(tdewfullpath,chunks={'time': 20})
        t2mvp = np.exp(1.81 + (ds_t2m.t2m * 17.27 - 4717.31) / (ds_t2m.t2m - 35.86))
        tdewvp = np.exp(1.81 + (ds_tdew.d2m * 17.27 - 4717.31) / (ds_tdew.d2m - 35.86))
        rh = (tdewvp / t2mvp) * 100
        # Create an output dataset with all attributes
        outds = ds_t2m
        outds = outds.drop('t2m')
        outds.attrs['Desc'] = "Derived from 2m air temperature and 2m dewpoint temperature from ERA5"
        rh.attrs['units'] = "%"
        rh.attrs['long_name'] = "2 metre relative humidity"
        outds['rh'] = rh
        outds = outds.groupby('time.dayofyear').max('time')
        ofile = t2mfullpath.replace('t2m','rhmax')
        print("Outfile: %s:" % (ofile))
        outds.to_netcdf(ofile,format= 'NETCDF4_CLASSIC' )
                
        
ProcessERA5RelativeHumidityMax()

2018
/media/BigFire/projects/era5/data/era5_single_levels_t2m_2018.nc
Outfile: /media/BigFire/projects/era5/data/era5_single_levels_rhmax_2018.nc:
2017
/media/BigFire/projects/era5/data/era5_single_levels_t2m_2017.nc
Outfile: /media/BigFire/projects/era5/data/era5_single_levels_rhmax_2017.nc:
2016
/media/BigFire/projects/era5/data/era5_single_levels_t2m_2016.nc
Outfile: /media/BigFire/projects/era5/data/era5_single_levels_rhmax_2016.nc:
2015
/media/BigFire/projects/era5/data/era5_single_levels_t2m_2015.nc
Outfile: /media/BigFire/projects/era5/data/era5_single_levels_rhmax_2015.nc:
2014
/media/BigFire/projects/era5/data/era5_single_levels_t2m_2014.nc
Outfile: /media/BigFire/projects/era5/data/era5_single_levels_rhmax_2014.nc:
2013
/media/BigFire/projects/era5/data/era5_single_levels_t2m_2013.nc
Outfile: /media/BigFire/projects/era5/data/era5_single_levels_rhmax_2013.nc:
2012
/media/BigFire/projects/era5/data/era5_single_levels_t2m_2012.nc
Outfile: /media/BigFire/projects/era5/data/era5_